# Panda Example

In [1]:
import sys
sys.path.append('lib')

import scipy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time, os
from IPython.display import clear_output
from IPython.core import display

import pybullet as p
import pybullet_data

import pinocchio as pin
from utils import *
from costs_pseudo import *
from functools import partial

DATA_PATH = '/home/teguh/git/rli/learning_distribution_gan/tf_robot_learning/data'

%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4, suppress=True)

## Part 2: Projection and Inverse Kinematics

### Setup Pybullet

In [2]:
physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

In [155]:
p.resetSimulation()

### Setup Robot & environment

In [156]:
robot_urdf = DATA_PATH + '/urdf/panda_arm.urdf'
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
mean_pose = 0.5*(joint_limits[0]+joint_limits[1])

plane_id = p.loadURDF('plane.urdf')
p.resetBasePositionAndOrientation(plane_id, (0,0,0.), (0,0,0,1))
# p.resetBasePositionAndOrientation(plane_id, (0,0,-.5), (0,0,0,1))

# table_square_id = p.loadURDF('table_square/table_square.urdf')
# p.resetBasePositionAndOrientation(table_square_id, (0.,0,-0.64), (0, 0, 0.7071068, 0.7071068))

# table_id = p.loadURDF('table/table.urdf')
# p.resetBasePositionAndOrientation(table_id, (.7,0,-0.5), (0, 0, 0.7071068, 0.7071068))

# shelf_urdf = DATA_PATH + '/urdf/bookshelf_simple_collision.urdf'
# shelf_id = p.loadURDF(fileName=shelf_urdf)
# p.resetBasePositionAndOrientation(shelf_id, (-0.6,0.6,-0.5), (0, 0, 0, 1.))

#for visualizing the desired target
_,_,ball_id = create_primitives(radius=0.05)

### Load model in pinocchio

In [158]:
robot_urdf = DATA_PATH + '/urdf/panda_arm.urdf';
rmodel = pin.buildModelFromUrdf(robot_urdf)
rdata = rmodel.createData()

pin_frame_names = [f.name for f in rmodel.frames]
ee_frame_id = rmodel.getFrameId('panda_hand2')

#### Define standard functions

In [159]:
computeJacobian_std = partial(computeJacobian, rmodel, rdata, ee_frame_id)
computePose_std = partial(computePose, rmodel, rdata, ee_frame_id)
set_q_std = partial(set_q,robot_id, pb_joint_indices)

***

## Using Cost Model for projection

In [160]:
rotation_ik = np.eye(3)
pose_cost = CostFrameTranslationFloatingBaseNew(rmodel, rdata, np.zeros(3), ee_frame_id, weight=np.array([1.,1.,1.]))
ori_cost = CostFrameRotationSE3FloatingBaseNew(rmodel, rdata, rotation_ik, ee_frame_id, weight=np.array([1.,1.,0.]))
bound_cost = CostBoundNew(joint_limits, 1e-4)
posture_cost = CostPostureNew(rmodel, rdata, mean_pose)

In [192]:
#for projection
cost_sum = CostSumNew(rmodel, rdata)
cost_sum.addCost(ori_cost, 20., 'ori_cost', 1e-4)
cost_sum.addCost(bound_cost, 20., 'joint_limit', 1e-3)

#for IK with horizontal gripper constraint
cost_sum_ik = CostSumNew(rmodel, rdata)
cost_sum_ik.addCost(pose_cost,50., 'pose_cost', 1e-3)
cost_sum_ik.addCost(ori_cost, 30., 'ori_cost', 1e-3)
cost_sum_ik.addCost(bound_cost, 30., 'joint_limit', 1e-3)
# cost_sum_ik.addCost(posture_cost, .001 , 'posture', 1e3)

robot_projector = TalosCostProjectorNew(cost_sum, rmodel, rdata, bounds = joint_limits)
robot_ik_solver = TalosCostProjectorNew(cost_sum_ik, rmodel, rdata, bounds = joint_limits)

In [193]:
class sampler():
    """
    General sampler, given the joint limits
    """

    def __init__(self, joint_limits=None):
        self.dof = joint_limits.shape[1]
        self.joint_limits = joint_limits

    def sample(self, N=1):
        samples = np.random.rand(N, self.dof)
        samples = self.joint_limits[0] + samples * (self.joint_limits[1] - self.joint_limits[0])
        return samples

In [194]:
workspace_area = np.array([[-0.6, -0.6 , -0.1], [0.6, 0.6, 1.0]])
target_sampler = sampler(workspace_area)
rob_simple_sampler = sampler(joint_limits)

#### Try the projection operator

In [195]:
q = rob_simple_sampler.sample().flatten()
res = robot_projector.project(q)
q, success, func_calls = res['q'], res['stat'], res['nfev']
set_q_std(q)

print(robot_projector.cost.calc_cost(q))

4.6077234534738006e-09


#### Compare IK

In [196]:
ik_workspace_area = np.array([[0.4, -0.5 , 0.5], [0.7, 0.5, .8]])
ik_target_sampler = sampler(ik_workspace_area)
goal_poses = ik_target_sampler.sample(1000)
for i in range(len(goal_poses)):
    goal_poses[i] = clip_bounds(goal_poses[i], ik_workspace_area)

#### Try IK

In [248]:
idx = np.random.randint(1000)
goal_pos = goal_poses[idx]
    
p.resetBasePositionAndOrientation(ball_id, goal_pos, np.array([0,0,0,1]))

robot_ik_solver.cost.costs['pose_cost'].cost.desired_pose = goal_pos
q = rob_simple_sampler.sample().flatten()
set_q_std(q)
print(robot_ik_solver.cost.calc_cost(q))
res = robot_ik_solver.project(q, maxiter = 100)
print(res['stat'], res['nfev'])
set_q_std(res['q'])

print(robot_ik_solver.cost.calc_cost(q))

331.95701434602574
False 100
14.193044287616594
